In [1]:
# First import the common Python packages
import numpy as np
# Pandas is like spreadsheets in python
import pandas as pd
# Lets get it to display all pandas columns
pd.set_option('display.max_columns', None)
# This is for plotting
import matplotlib.pyplot as plt

# import pyMELTScalc and locate the MELTS files
import pyMELTScalc as M
import sys

sys.path.append(r"MELTS")

# MELTS tutorial Q1 & 2 - liquidus, equilibrate, and crystallisation calculations

This notebook will take you through some simple calculations used to show how we might set up a crystallisation calculation for a high-silica rhyolite. The composition used is from the Oldest Toba Tuff. 

Above you should have loaded in the common Python packages, and told Python where to find the underlying alphaMELTS for Python code (in the sys.path.append(r' ') copy the location of the MELTS files on your PC).

First, lets set a starting composition. To do this we simply create a Python dictionary with the amount of each oxide.

In [2]:
comp = {'SiO2_Liq': 74.39,
        'TiO2_Liq': 0.18,
        'Al2O3_Liq': 13.55,
        'FeOt_Liq': 1.30,
        'MgO_Liq': 0.50,
        'CaO_Liq': 1.43,
        'Na2O_Liq': 3.36,
        'K2O_Liq': 5.09}

## Q1 - Liquidus and equilibrate calculations

To calculate the liquidus of the above melt composition at a set water content we can use the "findLiq_multi" function from pyMELTScalc. Try to calculate the liquidus temperature for the above melt composition at 1750 bars (175 MPa), fO$_2$ constrained at the Ni-NiO buffer, and a water content of 4 wt%:

In [3]:
import os
sys.stdout = open(os.devnull, 'w')
sys.stderr = open(os.devnull, 'w')

In [4]:
Results = M.findLiq_multi(Model = "MELTSv1.0.2", 
                        bulk = comp,
                        P_bar = 1750,
                        H2O_Liq = 8,
                        fO2_buffer = "NNO")    

This function will return a Dictionary containing the liquidus temperature (in Celsius), the liquidus phase, and the normalized melt composition (including volatile components).

In [5]:
Results

### Q1.1 What is the liquidus temperature?

-
-
-
-



### Q1.2 Keeping all other settings the same, what is the liquidus temperature of the magma when it is water saturated??

-
-
-

### Q1.3 What is the solubility of water in the melt at the liquidus?

-
-
-

### Q1.4 What is the liquidus phase?

-
-
-


**Use the code above to try and answer these questions. Try increasing the H$_2$O content until the melt is fluid saturated at the liquidus.**

Note that the pyMELTScalc output is normalised to a hydrous total of 100 wt%. By default, MELTS treats the input composition as grams of the different oxide values (e.g., grams of SiO$_2$, grams of TiO$_2$ etc.). How many grams of H$_2$O are required for the magma to be water saturated at the liquidus at 1750 bars?

This can be determined by multiplying the H$_2$O content in wt% by the ratio of the inputted SiO$_2$ content (in grams) and the outputted SiO$_2$ content (in wt%).

In [6]:
H2O_gram = Results['H2O']*comp['SiO2_Liq']/Results['SiO2']
H2O_gram

### Q1.5 Using the M.equilibrate_multi function, determine the mass percent of melt remaining in the system at 765$^o$C for a melt that is water-saturated (but contains no exsolved water) at the liquidus?**

In [7]:
Eq_Results = M.equilibrate_multi(Model = "MELTSv1.0.2", 
                            bulk = comp, 
                            T_C = 765, 
                            P_bar = 1750, 
                            H2O_Liq = 10, 
                            fO2_buffer = "NNO")

The results are all stored in a Dictionary called Eq_Results. Within this dictionary there is a DataFrame called 'Mass' (i.e., Eq_Results['Mass']) that contains  the mass of each phase at this pressure and temperature.

In [9]:
Eq_Results.keys()

### Q1.6 What is the Anorthite content of the plagioclase formed at this temperature?**

The composition of any phase can be accessed in the Eq_Results Dictionary by calling the DataFrame with the phase name followed by 1. i.e., for plagioclase compositions you would type Eq_Results['plagioclase1'].

In [10]:
Eq_Results['plagioclase1']

pyMELTScalc doesn't automatically give you the mineral proportions in terms of their end-member components. However, one of the great things about pyMELTScalc is that it's specifically designed to integrate with other Python packages. For example, here we can use Thermobar to calculate the Plagioclase cation fractions and end-member components without having to modify our output at all!

In [11]:
import Thermobar as pt
Plag = pt.calculate_cat_fractions_plagioclase(plag_comps = Eq_Results['plagioclase1'])
Plag['An_Plag']

### Q1.7 At what temperature does the mass of melt in the system drop below 10%?

-
-
-

## Q2 - Crystallisation calculations

### Q2.1 Using the same conditions and composition as above, run as isobaric cooling sequence (for now we will assume equilibrium crystallisation) with fO$_2$ constained at the Ni-NiO buffer. Start this simulation at the "wet liquidus" and continue until the system is more than 90% crystals!

For this, we will use a funcion termed M.isobaric_crystallisation

In this function we need to provide the pressure and fO$_2$ constraints as well as several other bits of information. Read the comments below to find out what each variable does:

In [12]:
Cryst_Results = M.isobaric_crystallisation(Model = "MELTSv1.0.2", # the version of MELTS to be used in the calculation
               bulk = comp, # starting composition as used previously
               T_start_C = 845, # temperature to start the calculation - use the wet liquidus temperature calculated above
               T_end_C = 750, # end temperature, likely won't be reached in this model but needed as a place holder
               dt_C = 0.5, # temperature intervals
               P_bar = 1750, # pressure (bars)
               H2O_Liq = 10, # grams of H2O in the system
               fO2_buffer = "NNO", # fO2 buffer
               fluid_sat = True, # if True the system will start at the wet liquidus, but remove any excess fluid prior to crystallisation
               Crystallinity_limit = 0.90) # Using this command tells the code to finish the model when the crystal fraction passes above the specified value

### Q2.2 At what temperature does feldspar saturate?

-
-
-

### Q2.3 What is the SiO$_2$ content at feldspar saturation?

-
-
-

### Q2.4 The composition of the first feldspar to saturate?

-
-
-


All outputs included within pyMELTScalc are saved in a pandas DataFrame called 'All'. We can simply select the parameters that we are interested in.